In [1]:
import pandas as pd
import numpy as np

Загружаем данные из ранее сохранённых файлов

In [2]:
post_text = pd.read_csv('post_text.csv')

In [3]:
post_stat = pd.read_csv('post_stat.csv')

In [4]:
user_data = pd.read_csv('user_data.csv')

In [5]:
user_stat = pd.read_csv('user_stat.csv')

In [6]:
feed_data = pd.read_csv('feed_data_100.csv')
feed_data.sort_values('timestamp', inplace=True, ignore_index=True)

В таблице feed_data есть колонка 'target', она будет являться целевой в нашем обучении.

0 - пост не получил like от пользователя, которому был показан.

1 - пост получил like.

При помощи метода TF-IDF в таблице post_text преобразуем текстовую колонку 'text' в векторный вид.
Из получившейся матрицы возьмём построчный максимум.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

post_text_copy = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(post_text['text'])
post_text_copy['text'] = F.toarray().max(axis=1)

Соединяем таблицы по колонкам post_id и user_id.

In [8]:
df = pd.merge(feed_data,
              post_text_copy,
              on='post_id',
              how='left')

In [9]:
df = pd.merge(df,
              post_stat,
              on='post_id',
              how='left')

In [10]:
df = pd.merge(df,
              user_data,
              on='user_id',
              how='left')

In [11]:
df = pd.merge(df,
              user_stat,
              on='user_id',
              how='left')

Из колонки timestamp выделяем дополнительные признаки: год, месяц, день, час, минута, секунда. А исходную колонку удаляем.

In [12]:
from datetime import datetime as dt

df['timestamp'] = df['timestamp'].astype('datetime64')

df['year'] = df['timestamp'].dt.year.astype('object')
df['month'] = df['timestamp'].dt.month.astype('object')
df['day'] = df['timestamp'].dt.day.astype('object')
df['hour'] = df['timestamp'].dt.hour.astype('object')
df['minute'] = df['timestamp'].dt.minute.astype('object')
df['second'] = df['timestamp'].dt.second.astype('object')

df = df.drop('timestamp', axis=1)

Делим сгруппированый набор данных на признаки "X" и целевую колонку "Y".

In [13]:
X = df.drop(['user_id', 'post_id', 'action', 'target'], axis=1)
y = df['target']

Делим колонки в таблице признаков на числовые и категориальные.

In [14]:
numeric_columns = list(X.select_dtypes(exclude='object').columns)
categorical_columns = list(X.select_dtypes(include='object').columns)

numeric_columns_ind = [list(X.columns).index(col) for col in numeric_columns]
columns_ohe_ind = [list(X.columns).index(col) for col in categorical_columns]

Создаём трансформер, который будет преобразовывать данные признаков следующим образом: числовые признаки стандартизирует, а к категориальным применит метод One-Hot.

In [15]:
from sklearn.preprocessing import StandardScaler
from category_encoders.one_hot import OneHotEncoder
from sklearn.compose import ColumnTransformer

t = [('StandardScaler', StandardScaler(), numeric_columns_ind),
     ('OneHotEncoder', OneHotEncoder(), columns_ohe_ind)]

columns_transformer = ColumnTransformer(transformers=t)

Делим данные на тренеровочные и тестовые.

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1)

Проверяем баланс классов на тренеровке и на тесте.

In [17]:
y_train.value_counts()[1] / len(y_train)

0.1566875

In [18]:
y_test.value_counts()[1] / len(y_test)

0.158

Стоит отметить, что в данных, на которых выполняется обучение, целевые классы имеют явный дисбаланс: класс 1 порядка 16%. При разбивке данных на тренеровочные и тестывые соотношение классов будем считать приемлемым.

При помощи Pipeline проводим обучение модели CatBoostClassifier. Параметры модели были подобранны вручную.

In [19]:
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

In [20]:
%%time

pipe = Pipeline([('columns_transformer', columns_transformer),
                 ('decision_tree', CatBoostClassifier(iterations = 10,
                                                      loss_function = 'MultiClass',
                                                      early_stopping_rounds=10,
                                                      thread_count = 100,
                                                      random_state=1))])
pipe.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.5116462	total: 170ms	remaining: 1.53s
1:	learn: 0.4520025	total: 189ms	remaining: 756ms
2:	learn: 0.4305365	total: 205ms	remaining: 478ms
3:	learn: 0.4213956	total: 221ms	remaining: 331ms
4:	learn: 0.4149518	total: 237ms	remaining: 237ms
5:	learn: 0.4119717	total: 253ms	remaining: 169ms
6:	learn: 0.4097816	total: 268ms	remaining: 115ms
7:	learn: 0.4073799	total: 284ms	remaining: 71ms
8:	learn: 0.4051426	total: 299ms	remaining: 33.2ms
9:	learn: 0.4027736	total: 315ms	remaining: 0us
Wall time: 7.89 s


Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  [0, 2, 3, 4, 7, 10]),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  [1, 5, 6, 8, 9, 11, 12, 13,
                                                   14, 15, 16])])),
                ('decision_tree',
                 <catboost.core.CatBoostClassifier object at 0x000001BB08A6AA00>)])

Сохраняем обученую модель.

In [21]:
import pickle

filename = 'cat_model.pkl'
pickle.dump(pipe, open(filename, 'wb'))

Рассчитаем ROC-AUC для обученной модели на тестовых данных.

In [22]:
from sklearn.metrics import roc_auc_score, accuracy_score

print(f'ROC-AUC {roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1])}')

ROC-AUC 0.6770286896028143


# Далее делаем пайплайн расчёта предсказаний
Для для рандомного пользователя и рандомного времени получим предсказания от ранее обученной модели, чтобы проверить её работоспособность.

In [24]:
note = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(note['text'])
note['text'] = F.toarray().max(axis=1)

In [25]:
note = pd.merge(note,
                post_stat,
                on='post_id',
                how='left')

In [26]:
users = pd.merge(user_data,
                 user_stat,
                 on='user_id',
                 how='left')

In [27]:
id = 345

In [28]:
time = dt(year=2021, month=10, day=30, hour=14)

In [29]:
limit = 10

In [30]:
def add_user_time(note, pers, time):
    note['gender'] = pers[0]
    note['age'] = pers[1]
    note['country'] = pers[2]
    note['city'] = pers[3]
    note['exp_group'] = pers[4]
    note['os'] = pers[5]
    note['source'] = pers[6]
    note['user_stat'] = pers[7]
    
    note['year'] = time.year
    note['month'] = time.month
    note['day'] = time.day
    note['hour'] = time.hour
    note['minute'] = time.minute
    note['second'] = time.second
    note[['year', 'month', 'day', 'hour', 'minute', 'second']] = note[['year', 'month', 'day', 'hour', 'minute', 'second']].astype('object')

    return note

In [31]:
%%time

pers = users[users['user_id']==id].drop('user_id', axis=1).iloc[0]

pred = add_user_time(note, pers, time).drop('post_id', axis=1)

predict = pd.DataFrame(post_text['post_id'])

predict['predict'] = pipe.predict_proba(pred)[:,1]

predict.sort_values('predict', ascending=False, inplace=True, ignore_index=True)

posts = predict.head(limit)['post_id'].values

posts = post_text[post_text['post_id'].isin(posts)]

posts

Wall time: 321 ms


,post_id,text,topic
4101,4281,What we have here is a film perfect for anyone...,movie
4214,4395,"Yes, I couldnt stop yawning, nor could my part...",movie
4378,4566,I feel very generous giving this movie a 2 out...,movie
4688,4889,"This movie starts out very VERY slow, but when...",movie
4786,4979,Even if you could get past the idea that these...,movie
4945,5137,"To be honest, I had no idea what this movie wa...",movie
5212,5423,The comments of the previous user are harsh in...,movie
5744,5982,I went to see this with my wife and 3 yr old s...,movie
5873,6119,A rather lame teen slasher from Brisbane. Whil...,movie
6818,7103,I knew I was in for a LONG 90 minutes when the...,movie
